In [ ]:
import os
import sys

# Fix directory if in notebooks folder
current_dir = os.getcwd()
if os.path.basename(current_dir) == 'notebooks':
    os.chdir(os.path.dirname(current_dir))
    print(f"✅ Changed to: {os.getcwd()}")

# Verify files
required_files = [
    'config/training_config.yaml',
    'data/processed/augmented_train.json',
    'src/trainer.py',
    'src/inference.py',
    'src/evaluation.py'
]

print("\n=== Verifying Files ===")
for file_path in required_files:
    exists = os.path.exists(file_path)
    print(f"{'✅' if exists else '❌'} {file_path}")
    if not exists:
        raise FileNotFoundError(f"{file_path} not found!")

print("\n✅ All files verified!")

In [ ]:
import sys
sys.path.append('src')

from src.trainer import (
    setup_model_and_tokenizer,
    apply_lora,
    prepare_dataset,
    train_model,
    save_checkpoint
)
from src.inference import translate_with_idioms, extract_idioms
from src.evaluation import evaluate_model, print_evaluation_report, save_predictions

import torch
import yaml
import json
from pathlib import Path
import shutil

print("✅ Imports successful!")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")

In [ ]:
with open('config/training_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("✅ Config loaded!")
print(f"Model: {config['model']['base_model']}")
print(f"Epochs: {config['training']['num_epochs']}")
print(f"Batch size: {config['training']['batch_size']}")
print(f"Learning rate: {config['training']['learning_rate']}")

In [ ]:
print("="*80)
print("LOADING MODEL")
print("="*80)

model, tokenizer = setup_model_and_tokenizer(
    model_name=config['model']['base_model'],
    special_tokens=None  # CRITICAL: No special tokens!
)

print(f"\n✓ Model loaded")
print(f"✓ Vocab size: {len(tokenizer)}")

# VERIFY - Must be 256204!
assert len(tokenizer) == 256204, f"ERROR: Vocab is {len(tokenizer)}, should be 256204!"
print("✅ Vocabulary size correct!")

# Verify language support
if hasattr(tokenizer, 'lang_code_to_id'):
    print(f"✓ {len(tokenizer.lang_code_to_id)} languages supported")
    print(f"✓ eng_Latn: {tokenizer.lang_code_to_id.get('eng_Latn', 'NOT FOUND')}")
    print(f"✓ sin_Sinh: {tokenizer.lang_code_to_id.get('sin_Sinh', 'NOT FOUND')}")

In [ ]:
# Test how idiom tags are tokenized
test_text = "He <IDIOM>kicked the bucket</IDIOM> yesterday."

tokenizer.src_lang = "eng_Latn"
tokens = tokenizer(test_text, return_tensors="pt")

print(f"Input: {test_text}")
print(f"Token IDs: {tokens['input_ids'][0][:15]}...")
print(f"Decoded: {tokenizer.decode(tokens['input_ids'][0])}")
print("\n✅ Idiom tags are tokenized as regular text!")

In [ ]:
model = apply_lora(model, config['lora'])
print("✅ LoRA applied!")

In [ ]:
checkpoint_dir = 'models/checkpoints'
if os.path.exists(checkpoint_dir):
    shutil.rmtree(checkpoint_dir, ignore_errors=True)
    print("🗑️  Old checkpoints deleted")
    
os.makedirs(checkpoint_dir, exist_ok=True)
print(f"✅ Clean directory: {checkpoint_dir}")

In [ ]:
train_dataset = prepare_dataset(
    data_path=config['data']['augmented_json'],
    tokenizer=tokenizer,
    src_lang=config['model']['source_lang'],
    tgt_lang=config['model']['target_lang'],
    max_length=config['training']['max_length']
)

print(f"✅ Dataset prepared: {len(train_dataset)} examples")

# Show sample
print(f"\nSample:")
print(f"  Input IDs: {train_dataset[0]['input_ids'][:10]}...")
print(f"  Labels: {train_dataset[0]['labels'][:10]}...")

In [ ]:
from pathlib import Path

output_dir = Path(config['paths']['checkpoints'])
output_dir.mkdir(parents=True, exist_ok=True)

print("\n" + "="*80)
print("STARTING TRAINING")
print("="*80)
print("\nExpected loss:")
print("  Initial: ~2-3 ✅")
print("  After 50 steps: ~1.8")
print("  Final: ~1.0-1.5")
print("\n⚠️  If loss > 10, there's an error!")
print("="*80 + "\n")

trained_model, trainer = train_model(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    config={**config['training'], **config['settings']},
    output_dir=str(output_dir)
)

print("\n✅ TRAINING COMPLETE!")

In [ ]:
final_model_path = config['paths']['final_model']
save_checkpoint(trained_model, tokenizer, final_model_path)

print(f"✅ Model saved to: {final_model_path}")

# Verify
if Path(final_model_path).exists():
    files = list(Path(final_model_path).glob('*'))
    print(f"   Saved {len(files)} files")

In [ ]:
print("="*80)
print("TEST TRANSLATIONS")
print("="*80 + "\n")

test_sentences = [
    "He <IDIOM>kicked the bucket</IDIOM> yesterday.",
    "She has a <IDIOM>green thumb</IDIOM> for gardening.",
    "It's <IDIOM>raining cats and dogs</IDIOM> outside.",
    "Don't <IDIOM>cry over spilled milk</IDIOM>.",
    "Let's <IDIOM>break the ice</IDIOM> with conversation."
]

translations = translate_with_idioms(
    model=trained_model,
    tokenizer=tokenizer,
    source_texts=test_sentences,
    src_lang=config['model']['source_lang'],
    tgt_lang=config['model']['target_lang'],
    num_beams=5
)

for i, (src, tgt) in enumerate(zip(test_sentences, translations), 1):
    print(f"{i}. EN: {src}")
    print(f"   SI: {tgt}")
    
    src_idioms = extract_idioms(src)
    tgt_idioms = extract_idioms(tgt)
    
    if src_idioms:
        print(f"   Source idioms: {src_idioms}")
    if tgt_idioms:
        print(f"   Target idioms: {tgt_idioms}")
    print()

print("="*80)

In [ ]:
print("="*80)
print("MODEL EVALUATION")
print("="*80 + "\n")

# Load test data
with open(config['data']['augmented_json'], 'r', encoding='utf-8') as f:
    all_data = json.load(f)

# Use subset for quick evaluation
test_data = all_data[:100]
print(f"Evaluating on {len(test_data)} examples...\n")

# Evaluate
metrics, predictions = evaluate_model(
    model=trained_model,
    tokenizer=tokenizer,
    test_data=test_data,
    src_lang=config['model']['source_lang'],
    tgt_lang=config['model']['target_lang']
)

# Print report
print_evaluation_report(metrics)

# Save predictions
os.makedirs('outputs', exist_ok=True)
save_predictions(
    predictions=predictions,
    references=[ex['target_si'] for ex in test_data],
    source_texts=[ex['source_en'] for ex in test_data],
    output_path='outputs/predictions.json'
)

print("\n✅ Evaluation complete!")
print("   Results saved to: outputs/predictions.json")

In [ ]:
# Load and display some predictions
with open('outputs/predictions.json', 'r', encoding='utf-8') as f:
    results = json.load(f)

print("="*80)
print("EXAMPLE PREDICTIONS")
print("="*80 + "\n")

# Show first 5 examples
for i, result in enumerate(results[:5], 1):
    print(f"Example {i}:")
    print(f"  Source: {result['source']}")
    print(f"  Prediction: {result['prediction']}")
    print(f"  Reference: {result['reference']}")
    
    if result['source_idioms']:
        print(f"  Source idioms: {result['source_idioms']}")
    if result['predicted_idioms']:
        print(f"  Predicted idioms: {result['predicted_idioms']}")
    if result['reference_idioms']:
        print(f"  Reference idioms: {result['reference_idioms']}")
    print()

print("="*80)

In [ ]:
# Calculate summary statistics from predictions
with open('outputs/predictions.json', 'r', encoding='utf-8') as f:
    results = json.load(f)

total_examples = len(results)
examples_with_idioms = sum(1 for r in results if r['source_idioms'])
idioms_correctly_translated = sum(
    1 for r in results 
    if r['source_idioms'] and r['predicted_idioms'] and 
    any(pred in ref for pred in r['predicted_idioms'] for ref in r['reference_idioms'])
)

print("="*80)
print("SUMMARY STATISTICS")
print("="*80 + "\n")

print(f"Total examples evaluated: {total_examples}")
print(f"Examples with idioms: {examples_with_idioms}")
print(f"Idioms with correct translation: {idioms_correctly_translated}")

if examples_with_idioms > 0:
    idiom_acc = (idioms_correctly_translated / examples_with_idioms) * 100
    print(f"\nIdiom translation accuracy: {idiom_acc:.2f}%")

print("\n" + "="*80)